In [1]:
# Copyright (c) 2024 Quan-feng WU <wuquanfeng@ihep.ac.cn>
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT

using DelimitedFiles
using FeynmanDenominators
using PakAlgorithm
using SymEngine

In [2]:
# Copyright (c) 2024 Quan-feng WU <wuquanfeng@ihep.ac.cn>
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT

include("generate_AMFlow_wolfram_script.jl")

generate_AMFlow_wolfram_script (generic function with 1 method)

In [3]:
# Copyright (c) 2024 Quan-feng WU <wuquanfeng@ihep.ac.cn>
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT

loop_momenta = ["k1", "k2"]
external_momenta = ["pa", "pb"]

den_list = map(den_mom -> FeynmanDenominator(den_mom, "m", 0; loop_momenta=loop_momenta, external_momenta=external_momenta), readlines("denMom.dat"))
exponent_mat = readdlm("exponentMat.dat", '\t', Int, '\n'; header=false)

den_collect_list = FeynmanDenominatorCollection[]

for exponent_list ∈ eachrow(exponent_mat)
    # any(iszero, exponent_list[1:3]) && continue
    den_indices = findall(!iszero, exponent_list)
    # println(den_indices)
    # (println ∘ string ∘ FeynmanDenominatorCollection)(loop_momenta, external_momenta, den_list[den_indices])
    push!(den_collect_list,
        FeynmanDenominatorCollection(loop_momenta, external_momenta, den_list[den_indices])
    )
end


In [4]:
# Copyright (c) 2024 Quan-feng WU <wuquanfeng@ihep.ac.cn>
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT

max_num_den, _ = findmax(length, den_collect_list)
@assert max_num_den == 7
max_den_collect_list = den_collect_list[findall(==(max_num_den) ∘ length, den_collect_list)]

containing_indices_list = [Int[] for _ ∈ eachindex(max_den_collect_list)]

for (ii, den_collect) ∈ enumerate(den_collect_list)
    ii_flag = false
    for (jj, max_den_collect) ∈ enumerate(max_den_collect_list)
        if den_collect ⊆ max_den_collect
            # @show exponent_mat[ii, :]
            push!(containing_indices_list[jj], ii)
            ii_flag = true
            break
        end
    end

    ii_flag &&
        @assert any(iszero, exponent_mat[ii, :]) """
        Denominator collection $ii is not contained in the maximal one:
        $(string(den_collect))
        """
end

# containing_indices_list

In [5]:
# Copyright (c) 2024 Quan-feng WU <wuquanfeng@ihep.ac.cn>
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT

open("wolfram_scripts/run.sh", "w+") do io
    write(io, "echo \"Start Time: \$(date +%H:%M:%S)\"\n")

    count = 0

    for (ii, max_den_collect) ∈ enumerate(max_den_collect_list)
        containing_indices = containing_indices_list[ii]
        sub_expontent_mat = exponent_mat[containing_indices, :]
        # (println ∘ string)(sub_expontent_mat)

        non_zero_col_list = findall(!iszero, eachcol(sub_expontent_mat))
        # @show length(non_zero_col_list)
        # @show non_zero_col_list
        @assert length(non_zero_col_list) == 7
        sub_expontent_mat = sub_expontent_mat[:, non_zero_col_list]
        # sub_expontent_mat = hcat(ones(Int, length(containing_indices), 3), sub_expontent_mat)

        indices_list = [collect(row) for row ∈ eachrow(sub_expontent_mat)]
        generate_AMFlow_wolfram_script(
            "wolfram_scripts/phase$ii", "phase$ii",
            max_den_collect, indices_list,
            100, 1
        )
        write(io, "wolframscript -f phase$ii.wls > phase$ii.log &\n")
        count += 1
        if count == 10
            write(io, "wait\n")
            count = 1
        end
    end

    write(io, "wait\n")
    write(io, "echo \"End Time: \$(date +%H:%M:%S)\"\n")
end

chmod("wolfram_scripts/run.sh", 0o755)

"wolfram_scripts/run.sh"

In [6]:
# Copyright (c) 2024 Quan-feng WU <wuquanfeng@ihep.ac.cn>
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT

